In [152]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [153]:
import pandas as pd

## Let's read Sasha's dataframe with country's macro features

In [154]:
df = pd.read_csv('filtered&merged.csv')
df.head()

,Unnamed: 0,Country Code,Year,Country Name,Researchers in R&D (per million people),GDP per Capita (USD),Research and development expenditure (% of GDP),Unemployemnt total (% of total labor force)
0,1,ARG,2015,Argentina,1231.4866,13679.626498,0.62262,7.579
1,2,ARG,2016,Argentina,1260.7010,12699.962314,0.55815,8.085
2,3,ARG,2017,Argentina,1212.4573,14532.500931,0.55631,8.347
3,4,ARG,2018,Argentina,1227.4042,11752.799892,0.48830,9.220
4,5,ARG,2019,Argentina,1231.5171,9955.974787,0.47813,9.843


In [155]:
print(df['Country Name'].unique())
print(df['Year'].unique())
countries = df['Country Name'].unique()

['Argentina' 'Austria' 'Belgium' 'Bulgaria' 'Canada' 'China' 'Cyprus'
 'Czechia' 'Germany' 'Denmark' 'Spain' 'Estonia' 'Finland' 'France'
 'Georgia' 'Croatia' 'Hungary' 'Ireland' 'Italy' 'Japan' 'Kazakhstan'
 'Kuwait' 'Lithuania' 'Luxembourg' 'Latvia' 'Mexico' 'North Macedonia'
 'Netherlands' 'Panama' 'Poland' 'Portugal' 'Romania' 'Russia' 'Singapore'
 'Serbia' 'Slovakia' 'Slovenia' 'Thailand' 'Turkey' 'Ukraine' 'Uruguay'
 'United States' 'Uzbekistan']
[2015 2016 2017 2018 2019 2020]


In [156]:
len(countries)

43

## Dependent variable - total count of country's residents patent applications 

In [157]:
patents = pd.read_csv('Patents.csv', skiprows=5, index_col = False)
patents.head()

,Origin,Origin (Code),Office,Type,2015,2016,2017,2018,2019,2020
0,Albania,AL,Total,Total,16.0,36.0,17.0,18.0,10.0,NaN
1,Algeria,DZ,Total,Total,106.0,112.0,157.0,162.0,119.0,173.0
2,Andorra,AD,Total,Total,NaN,15.0,42.0,14.0,44.0,45.0
3,Angola,AO,Total,Total,NaN,NaN,NaN,14.0,3.0,NaN
4,Antigua and Barbuda,AG,Total,Total,2.0,84.0,96.0,470.0,NaN,532.0


In [158]:
# In patents dataframe:
# - Czech Republic needs to be transformed to Czechia
# - Netherlands (Kingdom of the) needs to be transformed to Netherlands
# - Russian Federation needs to be changed to Russia
# - Türkiye must be changed to Turkey
# - United States of America need to be changed to United States
country_rename_map = {
    'Czech Republic': 'Czechia',
    'Netherlands (Kingdom of the)': 'Netherlands',
    'Russian Federation': 'Russia',
    'Türkiye': 'Turkey',
    'United States of America': 'United States'
}
patents['Origin'] = patents['Origin'].replace(country_rename_map)

In [159]:
patents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Origin         172 non-null    object 
 1   Origin (Code)  171 non-null    object 
 2   Office         172 non-null    object 
 3   Type           172 non-null    object 
 4   2015           142 non-null    float64
 5   2016           151 non-null    float64
 6   2017           146 non-null    float64
 7   2018           148 non-null    float64
 8   2019           137 non-null    float64
 9   2020           148 non-null    float64
dtypes: float64(6), object(4)
memory usage: 13.6+ KB


In [160]:
# Let's focus only on the countries from our list
patents_filtered = patents[patents['Origin'].isin(countries)]

In [161]:
len(patents_filtered['Origin'].unique())

43

In [162]:
patents_filtered[patents_filtered.isna().any(axis=1)]

,Origin,Origin (Code),Office,Type,2015,2016,2017,2018,2019,2020
78,Ireland,IE,Total,Total,NaN,4218.0,4234.0,5212.0,5286.0,5687.0
80,Italy,IT,Total,Total,NaN,25111.0,25150.0,26099.0,25719.0,26168.0
84,Kazakhstan,KZ,Total,Total,1413.0,1100.0,1181.0,944.0,NaN,918.0
86,Kuwait,KW,Total,Total,NaN,104.0,NaN,90.0,NaN,NaN
115,North Macedonia,MK,Total,Total,NaN,NaN,NaN,NaN,NaN,90.0
144,Slovenia,SI,Total,Total,NaN,NaN,NaN,582.0,NaN,679.0


In [163]:
# Ireland,Italy and Kazachstan can be filled in with mean value for their rows
# Kuwait, North MAcedonie and Slovenia can be dropped, as they have NaNs for most of the values

In [164]:
patents_filtered = patents_filtered[~patents_filtered['Origin'].isin(['Kuwait', 'North Macedonia', 'Slovenia'])]

In [165]:
# List of year columns (to operate only on numeric patent data)
year_cols = ['2015', '2016', '2017', '2018', '2019', '2020']

# For each row (i.e. each country), fill NaNs in year columns with the row's mean
patents_filtered[year_cols] = patents_filtered[year_cols].apply(
    lambda row: row.fillna(row.mean()), axis=1
)

In [166]:
patents_filtered.isna().any()

Origin           False
Origin (Code)    False
Office           False
Type             False
2015             False
2016             False
2017             False
2018             False
2019             False
2020             False
dtype: bool

## Melting the dataset

In [167]:
patents_filtered.head()

,Origin,Origin (Code),Office,Type,2015,2016,2017,2018,2019,2020
5,Argentina,AR,Total,Total,856.0,1111.0,742.0,702.0,774.0,1215.0
8,Austria,AT,Total,Total,11261.0,11118.0,10797.0,11280.0,11144.0,10645.0
15,Belgium,BE,Total,Total,8979.0,9485.0,10008.0,10437.0,10191.0,9665.0
24,Bulgaria,BG,Total,Total,467.0,389.0,380.0,394.0,380.0,517.0
30,Canada,CA,Total,Total,23258.0,22828.0,22252.0,22717.0,23504.0,22172.0


In [168]:
df

,Unnamed: 0,Country Code,Year,Country Name,Researchers in R&D (per million people),GDP per Capita (USD),Research and development expenditure (% of GDP),Unemployemnt total (% of total labor force)
0,1,ARG,2015,Argentina,1231.48660,13679.626498,0.62262,7.579
1,2,ARG,2016,Argentina,1260.70100,12699.962314,0.55815,8.085
2,3,ARG,2017,Argentina,1212.45730,14532.500931,0.55631,8.347
3,4,ARG,2018,Argentina,1227.40420,11752.799892,0.48830,9.220
4,5,ARG,2019,Argentina,1231.51710,9955.974787,0.47813,9.843
...,...,...,...,...,...,...,...,...
253,254,UZB,2016,Uzbekistan,509.89435,2753.379514,0.17238,5.160
254,255,UZB,2017,Uzbekistan,499.59238,2190.604116,0.14834,5.830
255,256,UZB,2018,Uzbekistan,480.43228,1813.085308,0.12394,5.389
256,257,UZB,2019,Uzbekistan,421.46490,2041.384807,0.11306,4.932


In [169]:
# Melt the patents dataframe
patents_long = patents_filtered.melt(
    id_vars=['Origin', 'Origin (Code)', 'Office', 'Type'],
    value_vars=['2015', '2016', '2017', '2018', '2019', '2020'],
    var_name='Year',
    value_name='Patent_Applications'
)

In [170]:
patents_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Origin               240 non-null    object 
 1   Origin (Code)        240 non-null    object 
 2   Office               240 non-null    object 
 3   Type                 240 non-null    object 
 4   Year                 240 non-null    object 
 5   Patent_Applications  240 non-null    float64
dtypes: float64(1), object(5)
memory usage: 11.4+ KB


In [171]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 8 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Unnamed: 0                                       258 non-null    int64  
 1   Country Code                                     258 non-null    object 
 2   Year                                             258 non-null    int64  
 3   Country Name                                     258 non-null    object 
 4   Researchers in R&D (per million people)          258 non-null    float64
 5   GDP per Capita (USD)                             258 non-null    float64
 6   Research and development expenditure (% of GDP)  258 non-null    float64
 7   Unemployemnt total (% of total labor force)      258 non-null    float64
dtypes: float64(4), int64(2), object(2)
memory usage: 16.3+ KB


In [172]:
df['Year'] = df['Year'].astype(str)

In [173]:
df.rename(columns={'Country Name': 'Origin'}, inplace=True)

In [174]:
panel = pd.merge(
    patents_long,
    df,
    on=['Origin', 'Year'],
    how='left'
)
panel = panel.drop(columns = ['Origin (Code)', 'Office', 'Type', 'Country Code'])
panel

,Origin,Year,Patent_Applications,Unnamed: 0,Researchers in R&D (per million people),GDP per Capita (USD),Research and development expenditure (% of GDP),Unemployemnt total (% of total labor force)
0,Argentina,2015,856.0,1,1231.48660,13679.626498,0.62262,7.579
1,Austria,2015,11261.0,7,5074.43000,43915.228021,3.04969,5.802
2,Belgium,2015,8979.0,13,4741.86040,40893.804538,2.42817,8.482
3,Bulgaria,2015,467.0,19,1939.14420,7078.860323,0.94947,9.143
4,Canada,2015,23258.0,25,4582.37940,43594.194105,1.69359,6.945
...,...,...,...,...,...,...,...,...
235,Turkey,2020,9353.0,234,1786.51710,8638.739038,1.36748,13.148
236,Ukraine,2020,1654.0,240,846.24850,3709.769287,0.40318,9.475
237,United States,2020,470131.0,252,4451.78120,64411.373178,3.42467,8.055
238,Uruguay,2020,94.0,246,813.23790,15789.685742,0.67468,10.413


## Final panel dataset containing Sasha's macro features + dependent variable
3 countries were dropped as the patents data was mostly unavailable for them

In [175]:
panel = panel.drop(columns = 'Unnamed: 0')
panel

,Origin,Year,Patent_Applications,Researchers in R&D (per million people),GDP per Capita (USD),Research and development expenditure (% of GDP),Unemployemnt total (% of total labor force)
0,Argentina,2015,856.0,1231.48660,13679.626498,0.62262,7.579
1,Austria,2015,11261.0,5074.43000,43915.228021,3.04969,5.802
2,Belgium,2015,8979.0,4741.86040,40893.804538,2.42817,8.482
3,Bulgaria,2015,467.0,1939.14420,7078.860323,0.94947,9.143
4,Canada,2015,23258.0,4582.37940,43594.194105,1.69359,6.945
...,...,...,...,...,...,...,...
235,Turkey,2020,9353.0,1786.51710,8638.739038,1.36748,13.148
236,Ukraine,2020,1654.0,846.24850,3709.769287,0.40318,9.475
237,United States,2020,470131.0,4451.78120,64411.373178,3.42467,8.055
238,Uruguay,2020,94.0,813.23790,15789.685742,0.67468,10.413


In [176]:
print(panel['Origin'].unique())
len(panel['Origin'].unique())

['Argentina' 'Austria' 'Belgium' 'Bulgaria' 'Canada' 'China' 'Croatia'
 'Cyprus' 'Czechia' 'Denmark' 'Estonia' 'Finland' 'France' 'Georgia'
 'Germany' 'Hungary' 'Ireland' 'Italy' 'Japan' 'Kazakhstan' 'Latvia'
 'Lithuania' 'Luxembourg' 'Mexico' 'Netherlands' 'Panama' 'Poland'
 'Portugal' 'Romania' 'Russia' 'Serbia' 'Singapore' 'Slovakia' 'Spain'
 'Thailand' 'Turkey' 'Ukraine' 'United States' 'Uruguay' 'Uzbekistan']


40

In [177]:
pip install pycountry

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [178]:
import pycountry

# Define a function to get the 3-letter country code
def get_country_code(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return None  # or you can put 'XXX' or anything for not found

# Create the new column
panel['Country Code'] = panel['Origin'].apply(get_country_code)


In [179]:
panel['Country Code'].unique()

array(['ARG', 'AUT', 'BEL', 'BGR', 'CAN', 'CHN', 'HRV', 'CYP', 'CZE',
       'DNK', 'EST', 'FIN', 'FRA', 'GEO', 'DEU', 'HUN', 'IRL', 'ITA',
       'JPN', 'KAZ', 'LVA', 'LTU', 'LUX', 'MEX', 'NLD', 'PAN', 'POL',
       'PRT', 'ROU', None, 'SRB', 'SGP', 'SVK', 'ESP', 'THA', 'UKR',
       'USA', 'URY', 'UZB'], dtype=object)

In [180]:
missing_codes = panel[panel['Country Code'].isnull()]
missing_codes

,Origin,Year,Patent_Applications,Researchers in R&D (per million people),GDP per Capita (USD),Research and development expenditure (% of GDP),Unemployemnt total (% of total labor force),Country Code
29,Russia,2015,31648.0,3109.7632,9277.713867,1.10085,5.615,None
35,Turkey,2015,6615.0,1205.0807,11049.995110,0.96732,10.304,None
69,Russia,2016,29373.0,2959.9658,8663.158203,1.10238,5.591,None
75,Turkey,2016,7583.0,1246.8931,10970.045895,1.11979,10.899,None
109,Russia,2017,25260.0,2825.5396,10658.913086,1.10967,5.238,None
115,Turkey,2017,10028.0,1369.3460,10695.550196,1.17632,10.919,None
149,Russia,2018,27689.0,2787.2463,11211.887695,0.99002,4.871,None
155,Turkey,2018,8609.0,1530.9047,9568.835063,1.27093,10.956,None
189,Russia,2019,26535.0,2749.4854,11447.701172,1.03531,4.513,None
195,Turkey,2019,9290.0,1629.7324,9215.440499,1.31960,13.730,None


In [181]:
panel.loc[panel['Origin'] == 'Russia', 'Country Code'] = panel.loc[panel['Origin'] == 'Russia', 'Country Code'].fillna('RUS')

In [182]:
panel.loc[panel['Origin'] == 'Turkey', 'Country Code'] = panel.loc[panel['Origin'] == 'Turkey', 'Country Code'].fillna('TUR')

In [183]:
panel['Country Code'].unique()

array(['ARG', 'AUT', 'BEL', 'BGR', 'CAN', 'CHN', 'HRV', 'CYP', 'CZE',
       'DNK', 'EST', 'FIN', 'FRA', 'GEO', 'DEU', 'HUN', 'IRL', 'ITA',
       'JPN', 'KAZ', 'LVA', 'LTU', 'LUX', 'MEX', 'NLD', 'PAN', 'POL',
       'PRT', 'ROU', 'RUS', 'SRB', 'SGP', 'SVK', 'ESP', 'THA', 'TUR',
       'UKR', 'USA', 'URY', 'UZB'], dtype=object)

In [184]:
panel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 8 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Origin                                           240 non-null    object 
 1   Year                                             240 non-null    object 
 2   Patent_Applications                              240 non-null    float64
 3   Researchers in R&D (per million people)          240 non-null    float64
 4   GDP per Capita (USD)                             240 non-null    float64
 5   Research and development expenditure (% of GDP)  240 non-null    float64
 6   Unemployemnt total (% of total labor force)      240 non-null    float64
 7   Country Code                                     240 non-null    object 
dtypes: float64(5), object(3)
memory usage: 15.1+ KB


In [185]:
panel['Year'] = panel['Year'].astype(int)


We have our panel dataset for period 2015-2020. Now, we can try to add independent variables such as first fillings, education, STEM graduates/capacities, Transfer Ratios*, Export Share, change in GDP. Actually - the variables that were used in the literature

In [186]:
indicators = pd.read_csv('indicator-data/indicators.csv')
indicators

,indicatorId,name,theme,lastDataUpdate,lastDataUpdateDescription,dataAvailability.totalRecordCount,dataAvailability.timeLine.min,dataAvailability.timeLine.max,dataAvailability.geoUnits.types,disaggregations,glossaryTerms
0,FOSGP.5T8.F600,Percentage of graduates from tertiary educatio...,EDUCATION,2025-02-23,February 2025 Data Release,1813,1998,2024,NATIONAL,"EduLvl:Tertiary, EduField:InfComm",2115.0
1,FOSGP.5T8.F700,Percentage of graduates from tertiary educatio...,EDUCATION,2025-02-23,February 2025 Data Release,1810,1998,2024,NATIONAL,"EduLvl:Tertiary, EduField:EngManCon",2115.0
2,MENF.5T8,Net flow of internationally mobile students (i...,EDUCATION,2025-02-23,February 2025 Data Release,2425,1998,2022,NATIONAL,NaN,3238.0
3,NY.GDP.MKTP.KD.ZG,GDP growth (annual %),DEMOGRAPHIC_SOCIOECONOMIC,2025-02-23,February 2025 Data Release,9937,1970,2023,NATIONAL,NaN,NaN
4,SP.POP.GROW,Population growth (annual %),DEMOGRAPHIC_SOCIOECONOMIC,2025-02-23,February 2025 Data Release,11641,1970,2023,NATIONAL,NaN,NaN
5,XGDP.FFNTR,"Expenditure on education (public, private, int...",EDUCATION,2025-02-23,February 2025 Data Release,333,2005,2023,NATIONAL,"FundingSource:Private, FundingSource:Public, F...",2150.0


In [187]:
data = pd.read_csv('indicator-data/data.csv')
data.head()

,indicatorId,geoUnit,year,value,qualifier,magnitude
0,FOSGP.5T8.F600,ABW,2015,0.00000,NaN,NIL
1,FOSGP.5T8.F600,ABW,2016,0.00000,NaN,NIL
2,FOSGP.5T8.F600,AFG,2020,4.58943,NaN,NaN
3,FOSGP.5T8.F600,AGO,2015,4.74076,NaN,NaN
4,FOSGP.5T8.F600,ALB,2015,5.05416,NaN,NaN


In [188]:
data['geoUnit'].unique()

array(['ABW', 'AFG', 'AGO', 'ALB', 'AND', 'ARE', 'ARG', 'ARM', 'AUS',
       'AUT', 'AZE', 'BDI', 'BEL', 'BEN', 'BFA', 'BGD', 'BGR', 'BHR',
       'BIH', 'BLR', 'BLZ', 'BMU', 'BRA', 'BRN', 'BWA', 'CAN', 'CHE',
       'CHL', 'CMR', 'COD', 'COG', 'COL', 'CPV', 'CRI', 'CUB', 'CYP',
       'CZE', 'DEU', 'DNK', 'DOM', 'DZA', 'ECU', 'EGY', 'ERI', 'ESP',
       'EST', 'FIN', 'FJI', 'FRA', 'GBR', 'GEO', 'GHA', 'GIB', 'GRC',
       'GRD', 'GTM', 'HND', 'HRV', 'HUN', 'IDN', 'IND', 'IRL', 'IRN',
       'ISL', 'ISR', 'ITA', 'JOR', 'KAZ', 'KGZ', 'KHM', 'KOR', 'LAO',
       'LIE', 'LKA', 'LSO', 'LTU', 'LUX', 'LVA', 'MAC', 'MAR', 'MCO',
       'MDA', 'MDG', 'MDV', 'MEX', 'MHL', 'MKD', 'MLT', 'MMR', 'MNE',
       'MNG', 'MOZ', 'MRT', 'MSR', 'MUS', 'MYS', 'NAM', 'NER', 'NIU',
       'NLD', 'NOR', 'NZL', 'OMN', 'PAN', 'PER', 'PHL', 'POL', 'PRI',
       'PRK', 'PRT', 'PSE', 'QAT', 'ROU', 'RUS', 'RWA', 'SAU', 'SDN',
       'SGP', 'SLV', 'SRB', 'SVK', 'SVN', 'SWE', 'SWZ', 'SXM', 'SYC',
       'SYR', 'TCD',

## GDP growth

In [189]:
# NY.GDP.MKTP.KD.ZG
gdp_growth = data[data['indicatorId'] == 'NY.GDP.MKTP.KD.ZG']
gdp_growth

,indicatorId,geoUnit,year,value,qualifier,magnitude
1817,NY.GDP.MKTP.KD.ZG,ABW,2015,-0.62363,NaN,NaN
1818,NY.GDP.MKTP.KD.ZG,ABW,2016,1.71962,NaN,NaN
1819,NY.GDP.MKTP.KD.ZG,ABW,2017,7.04853,NaN,NaN
1820,NY.GDP.MKTP.KD.ZG,ABW,2018,2.38173,NaN,NaN
1821,NY.GDP.MKTP.KD.ZG,ABW,2019,-2.30284,NaN,NaN
...,...,...,...,...,...,...
3060,NY.GDP.MKTP.KD.ZG,ZWE,2020,-7.81695,NaN,NaN
3061,NY.GDP.MKTP.KD.ZG,ZZA,2017,2.57442,NaN,NaN
3062,NY.GDP.MKTP.KD.ZG,ZZA,2018,2.06645,NaN,NaN
3063,NY.GDP.MKTP.KD.ZG,ZZA,2019,2.36074,NaN,NaN


In [190]:
gdp_growth = gdp_growth.drop(columns=['indicatorId', 'qualifier', 'magnitude'])
gdp_growth

,geoUnit,year,value
1817,ABW,2015,-0.62363
1818,ABW,2016,1.71962
1819,ABW,2017,7.04853
1820,ABW,2018,2.38173
1821,ABW,2019,-2.30284
...,...,...,...
3060,ZWE,2020,-7.81695
3061,ZZA,2017,2.57442
3062,ZZA,2018,2.06645
3063,ZZA,2019,2.36074


In [191]:
# Rename 'geoUnit' to 'Country Code' in gdp_growth dataframe
gdp_growth.rename(columns={'geoUnit': 'Country Code'}, inplace=True)
gdp_growth.rename(columns={'year':'Year'}, inplace=True)

# Merge the two dataframes based on 'Country Code' and 'Year'
panel_merged = pd.merge(panel, gdp_growth, on=['Country Code', 'Year'], how='left')

# Check the merged dataframe
panel_merged.head()


,Origin,Year,Patent_Applications,Researchers in R&D (per million people),GDP per Capita (USD),Research and development expenditure (% of GDP),Unemployemnt total (% of total labor force),Country Code,value
0,Argentina,2015,856.0,1231.4866,13679.626498,0.62262,7.579,ARG,2.73116
1,Austria,2015,11261.0,5074.4300,43915.228021,3.04969,5.802,AUT,1.01450
2,Belgium,2015,8979.0,4741.8604,40893.804538,2.42817,8.482,BEL,2.04146
3,Bulgaria,2015,467.0,1939.1442,7078.860323,0.94947,9.143,BGR,3.39755
4,Canada,2015,23258.0,4582.3794,43594.194105,1.69359,6.945,CAN,0.64997


In [192]:
panel_merged.rename(columns={'value':'gdp_growth'},inplace= True)

## Population growth

In [193]:
# SP.POP.GROW
pop_growth = data[data['indicatorId'] == 'SP.POP.GROW']
pop_growth

,indicatorId,geoUnit,year,value,qualifier,magnitude
3065,SP.POP.GROW,ABW,2015,0.63796,NaN,NaN
3066,SP.POP.GROW,ABW,2016,0.59006,NaN,NaN
3067,SP.POP.GROW,ABW,2017,0.53730,NaN,NaN
3068,SP.POP.GROW,ABW,2018,0.49480,NaN,NaN
3069,SP.POP.GROW,ABW,2019,0.45197,NaN,NaN
...,...,...,...,...,...,...
4356,SP.POP.GROW,ZZA,2016,0.93953,NaN,NaN
4357,SP.POP.GROW,ZZA,2017,0.90839,NaN,NaN
4358,SP.POP.GROW,ZZA,2018,1.22959,NaN,NaN
4359,SP.POP.GROW,ZZA,2019,1.27783,NaN,NaN


In [194]:
pop_growth.rename(columns={'geoUnit':'Country Code','year':'Year','value':'pop_growth'}, inplace=True)
pop_growth = pop_growth.drop(columns=['indicatorId','qualifier', 'magnitude'])
pop_growth.head()

C:\Users\grzes\AppData\Local\Temp\ipykernel_21232\3843614035.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_growth.rename(columns={'geoUnit':'Country Code','year':'Year','value':'pop_growth'}, inplace=True)


,Country Code,Year,pop_growth
3065,ABW,2015,0.63796
3066,ABW,2016,0.59006
3067,ABW,2017,0.53730
3068,ABW,2018,0.49480
3069,ABW,2019,0.45197


In [195]:
panel_merged = pd.merge(panel_merged,pop_growth,on=['Country Code',"Year"],how = 'left')
panel_merged.head()

,Origin,Year,Patent_Applications,Researchers in R&D (per million people),GDP per Capita (USD),Research and development expenditure (% of GDP),Unemployemnt total (% of total labor force),Country Code,gdp_growth,pop_growth
0,Argentina,2015,856.0,1231.4866,13679.626498,0.62262,7.579,ARG,2.73116,1.07800
1,Austria,2015,11261.0,5074.4300,43915.228021,3.04969,5.802,AUT,1.01450,1.12099
2,Belgium,2015,8979.0,4741.8604,40893.804538,2.42817,8.482,BEL,2.04146,0.57945
3,Bulgaria,2015,467.0,1939.1442,7078.860323,0.94947,9.143,BGR,3.39755,-0.63807
4,Canada,2015,23258.0,4582.3794,43594.194105,1.69359,6.945,CAN,0.64997,0.76030


## Net_flow

In [196]:
# MENF.5T8
net_flow = data[data['indicatorId'] == 'MENF.5T8']
net_flow.rename(columns={'geoUnit':'Country Code','year':'Year','value':'net_flow'}, inplace=True)
net_flow = net_flow.drop(columns=['indicatorId','qualifier', 'magnitude'])
net_flow.head()

C:\Users\grzes\AppData\Local\Temp\ipykernel_21232\4150712398.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  net_flow.rename(columns={'geoUnit':'Country Code','year':'Year','value':'net_flow'}, inplace=True)


,Country Code,Year,net_flow
1147,ABW,2015,99.0
1148,ABW,2016,89.0
1149,AFG,2020,-33310.0
1150,ALB,2018,-16791.0
1151,ALB,2019,-8850.0


In [197]:
panel_merged = pd.merge(panel_merged, net_flow, on=['Country Code','Year'],how='left')
panel_merged

,Origin,Year,Patent_Applications,Researchers in R&D (per million people),GDP per Capita (USD),Research and development expenditure (% of GDP),Unemployemnt total (% of total labor force),Country Code,gdp_growth,pop_growth,net_flow
0,Argentina,2015,856.0,1231.48660,13679.626498,0.62262,7.579,ARG,2.73116,1.07800,NaN
1,Austria,2015,11261.0,5074.43000,43915.228021,3.04969,5.802,AUT,1.01450,1.12099,50254.347656
2,Belgium,2015,8979.0,4741.86040,40893.804538,2.42817,8.482,BEL,2.04146,0.57945,42932.000000
3,Bulgaria,2015,467.0,1939.14420,7078.860323,0.94947,9.143,BGR,3.39755,-0.63807,-12491.000000
4,Canada,2015,23258.0,4582.37940,43594.194105,1.69359,6.945,CAN,0.64997,0.76030,121634.000000
...,...,...,...,...,...,...,...,...,...,...,...
235,Turkey,2020,9353.0,1786.51710,8638.739038,1.36748,13.148,TUR,1.85984,0.97039,133756.000000
236,Ukraine,2020,1654.0,846.24850,3709.769287,0.40318,9.475,UKR,-3.75280,-0.60161,-8580.000000
237,United States,2020,470131.0,4451.78120,64411.373178,3.42467,8.055,USA,-2.21347,0.96900,848728.187500
238,Uruguay,2020,94.0,813.23790,15789.685742,0.67468,10.413,URY,-7.38010,0.01974,-2530.000000


## Expenditure on education

In [198]:
# XGDP.FFNTR	
edu_exp = data[data['indicatorId'] == 'XGDP.FFNTR']
edu_exp.head()

,indicatorId,geoUnit,year,value,qualifier,magnitude
4361,XGDP.FFNTR,AND,2015,3.801682,NaN,NaN
4362,XGDP.FFNTR,AND,2016,3.808137,NaN,NaN
4363,XGDP.FFNTR,AND,2017,3.762174,NaN,NaN
4364,XGDP.FFNTR,AND,2018,3.796796,NaN,NaN
4365,XGDP.FFNTR,AND,2019,3.729350,NaN,NaN


In [199]:
edu_exp.rename(columns={'geoUnit':'Country Code','year':'Year','value':'edu_exp'}, inplace=True)
edu_exp = edu_exp.drop(columns=['indicatorId','qualifier', 'magnitude'])
edu_exp.head()

C:\Users\grzes\AppData\Local\Temp\ipykernel_21232\1999689733.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edu_exp.rename(columns={'geoUnit':'Country Code','year':'Year','value':'edu_exp'}, inplace=True)


,Country Code,Year,edu_exp
4361,AND,2015,3.801682
4362,AND,2016,3.808137
4363,AND,2017,3.762174
4364,AND,2018,3.796796
4365,AND,2019,3.729350


In [200]:
panel_merged = pd.merge(panel_merged, edu_exp, on=['Country Code','Year'],how='left')
panel_merged

,Origin,Year,Patent_Applications,Researchers in R&D (per million people),GDP per Capita (USD),Research and development expenditure (% of GDP),Unemployemnt total (% of total labor force),Country Code,gdp_growth,pop_growth,net_flow,edu_exp
0,Argentina,2015,856.0,1231.48660,13679.626498,0.62262,7.579,ARG,2.73116,1.07800,NaN,NaN
1,Austria,2015,11261.0,5074.43000,43915.228021,3.04969,5.802,AUT,1.01450,1.12099,50254.347656,5.848230
2,Belgium,2015,8979.0,4741.86040,40893.804538,2.42817,8.482,BEL,2.04146,0.57945,42932.000000,7.123384
3,Bulgaria,2015,467.0,1939.14420,7078.860323,0.94947,9.143,BGR,3.39755,-0.63807,-12491.000000,NaN
4,Canada,2015,23258.0,4582.37940,43594.194105,1.69359,6.945,CAN,0.64997,0.76030,121634.000000,5.815642
...,...,...,...,...,...,...,...,...,...,...,...,...
235,Turkey,2020,9353.0,1786.51710,8638.739038,1.36748,13.148,TUR,1.85984,0.97039,133756.000000,NaN
236,Ukraine,2020,1654.0,846.24850,3709.769287,0.40318,9.475,UKR,-3.75280,-0.60161,-8580.000000,NaN
237,United States,2020,470131.0,4451.78120,64411.373178,3.42467,8.055,USA,-2.21347,0.96900,848728.187500,7.402097
238,Uruguay,2020,94.0,813.23790,15789.685742,0.67468,10.413,URY,-7.38010,0.01974,-2530.000000,NaN


## STEM graduates

In [201]:
# Percentage of graduates from tertiary education graduating from Information and Communication Technologies programmes
# FOSGP.5T8.F600	
informatics = data[data['indicatorId'] == 'FOSGP.5T8.F600']
informatics.rename(columns={'geoUnit':'Country Code','year':'Year','value':'edu_exp'}, inplace=True)
informatics = informatics.drop(columns=['indicatorId','qualifier', 'magnitude'])
informatics.head()

C:\Users\grzes\AppData\Local\Temp\ipykernel_21232\2322452174.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  informatics.rename(columns={'geoUnit':'Country Code','year':'Year','value':'edu_exp'}, inplace=True)


,Country Code,Year,edu_exp
0,ABW,2015,0.00000
1,ABW,2016,0.00000
2,AFG,2020,4.58943
3,AGO,2015,4.74076
4,ALB,2015,5.05416


In [202]:
informatics.rename(columns={'edu_exp': 'inf_num'}, inplace=True)

In [203]:
informatics.head()

,Country Code,Year,inf_num
0,ABW,2015,0.00000
1,ABW,2016,0.00000
2,AFG,2020,4.58943
3,AGO,2015,4.74076
4,ALB,2015,5.05416


In [204]:
# Percentage of graduates from tertiary education graduating from Engineering, Manufacturing and Construction  programmes, both sexes
engineers = data[data['indicatorId'] == 'FOSGP.5T8.F700']
engineers.rename(columns={'geoUnit':'Country Code','year':'Year','value':'eng_num'}, inplace=True)
engineers = engineers.drop(columns=['indicatorId','qualifier', 'magnitude'])
engineers.head()

C:\Users\grzes\AppData\Local\Temp\ipykernel_21232\3637763681.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  engineers.rename(columns={'geoUnit':'Country Code','year':'Year','value':'eng_num'}, inplace=True)


,Country Code,Year,eng_num
573,ABW,2015,0.00000
574,ABW,2016,0.00000
575,AFG,2020,5.84207
576,AGO,2015,4.56635
577,ALB,2015,7.80828


In [205]:
# Additionally, let's read one more feature
stats = pd.read_csv('data2.csv')
stats.rename(columns={'geoUnit':'Country Code','year':'Year','value':'stats_num'}, inplace=True)
stats = stats.drop(columns=['indicatorId','qualifier', 'magnitude'])
stats.head()

,Country Code,Year,stats_num
0,ABW,2015,0.00000
1,ABW,2016,2.22222
2,AFG,2020,0.81362
3,AGO,2015,2.70335
4,ALB,2015,5.28268


In [206]:
# Let's add the values to obtain the total number of STEM students
step1 = pd.merge(informatics,engineers,on=['Country Code','Year'],how='left')
step1.head()

,Country Code,Year,inf_num,eng_num
0,ABW,2015,0.00000,0.00000
1,ABW,2016,0.00000,0.00000
2,AFG,2020,4.58943,5.84207
3,AGO,2015,4.74076,4.56635
4,ALB,2015,5.05416,7.80828


In [207]:
step2 = pd.merge(step1,stats,on=['Country Code','Year'],how='left')
step2.head()

,Country Code,Year,inf_num,eng_num,stats_num
0,ABW,2015,0.00000,0.00000,0.00000
1,ABW,2016,0.00000,0.00000,2.22222
2,AFG,2020,4.58943,5.84207,0.81362
3,AGO,2015,4.74076,4.56635,2.70335
4,ALB,2015,5.05416,7.80828,5.28268


In [208]:
step2['stem_graduates'] = step2['eng_num'] + step2['inf_num'] + step2['stats_num']
step2 = step2.drop(columns=['inf_num','eng_num','stats_num'])
step2.head()

,Country Code,Year,stem_graduates
0,ABW,2015,0.00000
1,ABW,2016,2.22222
2,AFG,2020,11.24512
3,AGO,2015,12.01046
4,ALB,2015,18.14512


## We finished the dataset. Let's look at its missing values

In [209]:
panel_merged = pd.merge(panel_merged,step2,on=['Country Code','Year'],how='left')
panel_merged.head()

,Origin,Year,Patent_Applications,Researchers in R&D (per million people),GDP per Capita (USD),Research and development expenditure (% of GDP),Unemployemnt total (% of total labor force),Country Code,gdp_growth,pop_growth,net_flow,edu_exp,stem_graduates
0,Argentina,2015,856.0,1231.4866,13679.626498,0.62262,7.579,ARG,2.73116,1.07800,NaN,NaN,NaN
1,Austria,2015,11261.0,5074.4300,43915.228021,3.04969,5.802,AUT,1.01450,1.12099,50254.347656,5.848230,29.271710
2,Belgium,2015,8979.0,4741.8604,40893.804538,2.42817,8.482,BEL,2.04146,0.57945,42932.000000,7.123384,17.382830
3,Bulgaria,2015,467.0,1939.1442,7078.860323,0.94947,9.143,BGR,3.39755,-0.63807,-12491.000000,NaN,20.793081
4,Canada,2015,23258.0,4582.3794,43594.194105,1.69359,6.945,CAN,0.64997,0.76030,121634.000000,5.815642,20.027500


In [210]:
panel_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 13 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Origin                                           240 non-null    object 
 1   Year                                             240 non-null    int64  
 2   Patent_Applications                              240 non-null    float64
 3   Researchers in R&D (per million people)          240 non-null    float64
 4   GDP per Capita (USD)                             240 non-null    float64
 5   Research and development expenditure (% of GDP)  240 non-null    float64
 6   Unemployemnt total (% of total labor force)      240 non-null    float64
 7   Country Code                                     240 non-null    object 
 8   gdp_growth                                       240 non-null    float64
 9   pop_growth                      

In [211]:
# panel_merged.to_csv('panel_merged.csv', index=False)

In [212]:
panel_merged[panel_merged['edu_exp'].isna()]['Origin'].value_counts()

Origin
Argentina     6
China         6
France        6
Estonia       6
Kazakhstan    6
Japan         6
Georgia       6
Hungary       6
Mexico        6
Singapore     6
Serbia        6
Panama        6
Ukraine       6
Uruguay       6
Uzbekistan    6
Thailand      6
Germany       5
Russia        5
Croatia       5
Canada        5
Turkey        5
Ireland       3
Bulgaria      2
Denmark       1
Portugal      1
Luxembourg    1
Slovakia      1
Cyprus        1
Name: count, dtype: int64

In [213]:
# Dobra, education expenses do dropniecia

In [214]:
export = pd.read_csv('export_share_data.csv')
export

,IndicatorCode,IndicatorName,VariableName,MeasurementName,CountryCode,Alpha3Code,CountryName,PeriodCode,Value,Unnamed: 9
0,4,"Share of exports in GDP, %",Share of exports in GDP,Percent,8,ALB,Albania,2005,23.0,NaN
1,4,"Share of exports in GDP, %",Share of exports in GDP,Percent,8,ALB,Albania,2006,25.2,NaN
2,4,"Share of exports in GDP, %",Share of exports in GDP,Percent,8,ALB,Albania,2007,28.2,NaN
3,4,"Share of exports in GDP, %",Share of exports in GDP,Percent,8,ALB,Albania,2008,25.3,NaN
4,4,"Share of exports in GDP, %",Share of exports in GDP,Percent,8,ALB,Albania,2009,25.2,NaN
...,...,...,...,...,...,...,...,...,...,...
1115,4,"Share of exports in GDP, %",Share of exports in GDP,Percent,860,UZB,Uzbekistan,2020,21.9,NaN
1116,4,"Share of exports in GDP, %",Share of exports in GDP,Percent,860,UZB,Uzbekistan,2021,21.3,NaN
1117,4,"Share of exports in GDP, %",Share of exports in GDP,Percent,860,UZB,Uzbekistan,2022,24.3,NaN
1118,4,"Share of exports in GDP, %",Share of exports in GDP,Percent,860,UZB,Uzbekistan,2023,23.7,NaN


In [215]:
export.rename(columns={'Alpha3Code':'Origin'}, inplace= True)

In [216]:
export.rename(columns={'PeriodCode':'Year'}, inplace= True)

In [217]:
export

,IndicatorCode,IndicatorName,VariableName,MeasurementName,CountryCode,Origin,CountryName,Year,Value,Unnamed: 9
0,4,"Share of exports in GDP, %",Share of exports in GDP,Percent,8,ALB,Albania,2005,23.0,NaN
1,4,"Share of exports in GDP, %",Share of exports in GDP,Percent,8,ALB,Albania,2006,25.2,NaN
2,4,"Share of exports in GDP, %",Share of exports in GDP,Percent,8,ALB,Albania,2007,28.2,NaN
3,4,"Share of exports in GDP, %",Share of exports in GDP,Percent,8,ALB,Albania,2008,25.3,NaN
4,4,"Share of exports in GDP, %",Share of exports in GDP,Percent,8,ALB,Albania,2009,25.2,NaN
...,...,...,...,...,...,...,...,...,...,...
1115,4,"Share of exports in GDP, %",Share of exports in GDP,Percent,860,UZB,Uzbekistan,2020,21.9,NaN
1116,4,"Share of exports in GDP, %",Share of exports in GDP,Percent,860,UZB,Uzbekistan,2021,21.3,NaN
1117,4,"Share of exports in GDP, %",Share of exports in GDP,Percent,860,UZB,Uzbekistan,2022,24.3,NaN
1118,4,"Share of exports in GDP, %",Share of exports in GDP,Percent,860,UZB,Uzbekistan,2023,23.7,NaN


In [226]:
print(len(export["Origin"].unique()))
export["Origin"].unique()

56


array(['ALB', 'AND', 'ARM', 'AUT', 'AZE', 'BLR', 'BEL', 'BIH', 'BGR',
       'CAN', 'HRV', 'CYP', 'CZE', 'DNK', 'EST', 'FIN', 'FRA', 'GEO',
       'DEU', 'GRC', 'HUN', 'ISL', 'IRL', 'ISR', 'ITA', 'KAZ', 'KGZ',
       'LVA', 'LIE', 'LTU', 'LUX', 'MLT', 'MCO', 'MNE', 'NLD', 'MKD',
       'NOR', 'POL', 'PRT', 'MDA', 'ROU', 'RUS', 'SMR', 'SRB', 'SVK',
       'SVN', 'ESP', 'SWE', 'CHE', 'TJK', 'TUR', 'TKM', 'UKR', 'GBR',
       'USA', 'UZB'], dtype=object)

In [220]:
new_panel = panel_merged.merge(
    export[['Origin', 'Year', 'Value']],
    on=['Origin', 'Year'],
    how='left'
)


In [221]:
new_panel

,Origin,Year,Patent_Applications,Researchers in R&D (per million people),GDP per Capita (USD),Research and development expenditure (% of GDP),Unemployemnt total (% of total labor force),Country Code,gdp_growth,pop_growth,net_flow,edu_exp,stem_graduates,Value
0,Argentina,2015,856.0,1231.48660,13679.626498,0.62262,7.579,ARG,2.73116,1.07800,NaN,NaN,NaN,NaN
1,Austria,2015,11261.0,5074.43000,43915.228021,3.04969,5.802,AUT,1.01450,1.12099,50254.347656,5.848230,29.271710,NaN
2,Belgium,2015,8979.0,4741.86040,40893.804538,2.42817,8.482,BEL,2.04146,0.57945,42932.000000,7.123384,17.382830,NaN
3,Bulgaria,2015,467.0,1939.14420,7078.860323,0.94947,9.143,BGR,3.39755,-0.63807,-12491.000000,NaN,20.793081,NaN
4,Canada,2015,23258.0,4582.37940,43594.194105,1.69359,6.945,CAN,0.64997,0.76030,121634.000000,5.815642,20.027500,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,Turkey,2020,9353.0,1786.51710,8638.739038,1.36748,13.148,TUR,1.85984,0.97039,133756.000000,NaN,15.212980,NaN
236,Ukraine,2020,1654.0,846.24850,3709.769287,0.40318,9.475,UKR,-3.75280,-0.60161,-8580.000000,NaN,24.254510,NaN
237,United States,2020,470131.0,4451.78120,64411.373178,3.42467,8.055,USA,-2.21347,0.96900,848728.187500,7.402097,19.553050,NaN
238,Uruguay,2020,94.0,813.23790,15789.685742,0.67468,10.413,URY,-7.38010,0.01974,-2530.000000,NaN,15.237220,NaN


In [219]:
new_panel = pd.merge(panel_merged,export,on=['Year','Origin'],how='left')
new_panel

,Origin,Year,Patent_Applications,Researchers in R&D (per million people),GDP per Capita (USD),Research and development expenditure (% of GDP),Unemployemnt total (% of total labor force),Country Code,gdp_growth,pop_growth,...,edu_exp,stem_graduates,IndicatorCode,IndicatorName,VariableName,MeasurementName,CountryCode,CountryName,Value,Unnamed: 9
0,Argentina,2015,856.0,1231.48660,13679.626498,0.62262,7.579,ARG,2.73116,1.07800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Austria,2015,11261.0,5074.43000,43915.228021,3.04969,5.802,AUT,1.01450,1.12099,...,5.848230,29.271710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Belgium,2015,8979.0,4741.86040,40893.804538,2.42817,8.482,BEL,2.04146,0.57945,...,7.123384,17.382830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bulgaria,2015,467.0,1939.14420,7078.860323,0.94947,9.143,BGR,3.39755,-0.63807,...,NaN,20.793081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Canada,2015,23258.0,4582.37940,43594.194105,1.69359,6.945,CAN,0.64997,0.76030,...,5.815642,20.027500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,Turkey,2020,9353.0,1786.51710,8638.739038,1.36748,13.148,TUR,1.85984,0.97039,...,NaN,15.212980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,Ukraine,2020,1654.0,846.24850,3709.769287,0.40318,9.475,UKR,-3.75280,-0.60161,...,NaN,24.254510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237,United States,2020,470131.0,4451.78120,64411.373178,3.42467,8.055,USA,-2.21347,0.96900,...,7.402097,19.553050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,Uruguay,2020,94.0,813.23790,15789.685742,0.67468,10.413,URY,-7.38010,0.01974,...,NaN,15.237220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [229]:
pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [230]:
export2 = pd.read_excel('WITS-Country-Timeseries.xlsx')
export2

,Country Name,Indicator Name,2015,2016,2017,2018,2019,2020
0,Afghanistan,Exports of goods and services (% of GDP),NaN,NaN,NaN,NaN,NaN,NaN
1,Albania,Exports of goods and services (% of GDP),27.267391,28.977899,31.569798,31.571954,31.304270,22.657790
2,Algeria,Exports of goods and services (% of GDP),23.171778,20.872485,22.632232,25.861151,22.714404,17.469840
3,Andorra,Exports of goods and services (% of GDP),NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,Exports of goods and services (% of GDP),29.754598,28.124485,29.004102,40.836290,40.790755,38.038274
...,...,...,...,...,...,...,...,...
188,Venezuela,Exports of goods and services (% of GDP),NaN,NaN,NaN,NaN,NaN,NaN
189,Vietnam,Exports of goods and services (% of GDP),72.922848,74.107286,81.762516,84.423458,85.157590,84.381595
190,Yemen,Exports of goods and services (% of GDP),18.321800,5.977269,8.709109,6.056570,NaN,NaN
191,Zambia,Exports of goods and services (% of GDP),37.138526,35.324676,34.992858,37.957405,34.636193,46.790165


In [231]:
import pandas as pd
import pycountry

# Function to convert country name to 3-letter ISO code
def get_country_code(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except LookupError:
        return None  # or custom handling for unmatched names

# Apply the function to your 'Country Name' column
export2['Country Code'] = export2['Country Name'].apply(get_country_code)


In [233]:
export2

,Country Name,Indicator Name,2015,2016,2017,2018,2019,2020,Country Code
0,Afghanistan,Exports of goods and services (% of GDP),NaN,NaN,NaN,NaN,NaN,NaN,AFG
1,Albania,Exports of goods and services (% of GDP),27.267391,28.977899,31.569798,31.571954,31.304270,22.657790,ALB
2,Algeria,Exports of goods and services (% of GDP),23.171778,20.872485,22.632232,25.861151,22.714404,17.469840,DZA
3,Andorra,Exports of goods and services (% of GDP),NaN,NaN,NaN,NaN,NaN,NaN,AND
4,Angola,Exports of goods and services (% of GDP),29.754598,28.124485,29.004102,40.836290,40.790755,38.038274,AGO
...,...,...,...,...,...,...,...,...,...
188,Venezuela,Exports of goods and services (% of GDP),NaN,NaN,NaN,NaN,NaN,NaN,VEN
189,Vietnam,Exports of goods and services (% of GDP),72.922848,74.107286,81.762516,84.423458,85.157590,84.381595,VNM
190,Yemen,Exports of goods and services (% of GDP),18.321800,5.977269,8.709109,6.056570,NaN,NaN,YEM
191,Zambia,Exports of goods and services (% of GDP),37.138526,35.324676,34.992858,37.957405,34.636193,46.790165,ZMB


In [236]:
# Melt the dataframe
export_long = pd.melt(
    export2,
    id_vars=['Country Name', 'Indicator Name', 'Country Code'],
    value_vars=['2015', '2016', '2017', '2018', '2019', '2020'],
    var_name='Year',
    value_name='Export_Share_GDP'
)

# Convert 'Year' to integer
export_long['Year'] = export_long['Year'].astype(int)


In [246]:
export_long = export_long.rename(columns={'Country Code':"Origin"})

In [256]:
export_long

,Country Name,Indicator Name,Origin,Year,Export_Share_GDP
0,Afghanistan,Exports of goods and services (% of GDP),AFG,2015,NaN
1,Albania,Exports of goods and services (% of GDP),ALB,2015,27.267391
2,Algeria,Exports of goods and services (% of GDP),DZA,2015,23.171778
3,Andorra,Exports of goods and services (% of GDP),AND,2015,NaN
4,Angola,Exports of goods and services (% of GDP),AGO,2015,29.754598
...,...,...,...,...,...
1153,Venezuela,Exports of goods and services (% of GDP),VEN,2020,NaN
1154,Vietnam,Exports of goods and services (% of GDP),VNM,2020,84.381595
1155,Yemen,Exports of goods and services (% of GDP),YEM,2020,NaN
1156,Zambia,Exports of goods and services (% of GDP),ZMB,2020,46.790165


In [261]:
export_long = export_long.rename(columns={'Origin':'Country Code'})

In [263]:
new_panel = panel_merged.merge(export_long,on=['Country Code','Year'],how='left')
new_panel

,Origin,Year,Patent_Applications,Researchers in R&D (per million people),GDP per Capita (USD),Research and development expenditure (% of GDP),Unemployemnt total (% of total labor force),Country Code,gdp_growth,pop_growth,net_flow,edu_exp,stem_graduates,Country Name,Indicator Name,Export_Share_GDP
0,Argentina,2015,856.0,1231.48660,13679.626498,0.62262,7.579,ARG,2.73116,1.07800,NaN,NaN,NaN,Argentina,Exports of goods and services (% of GDP),10.705652
1,Austria,2015,11261.0,5074.43000,43915.228021,3.04969,5.802,AUT,1.01450,1.12099,50254.347656,5.848230,29.271710,Austria,Exports of goods and services (% of GDP),53.089061
2,Belgium,2015,8979.0,4741.86040,40893.804538,2.42817,8.482,BEL,2.04146,0.57945,42932.000000,7.123384,17.382830,Belgium,Exports of goods and services (% of GDP),77.805282
3,Bulgaria,2015,467.0,1939.14420,7078.860323,0.94947,9.143,BGR,3.39755,-0.63807,-12491.000000,NaN,20.793081,Bulgaria,Exports of goods and services (% of GDP),63.800087
4,Canada,2015,23258.0,4582.37940,43594.194105,1.69359,6.945,CAN,0.64997,0.76030,121634.000000,5.815642,20.027500,Canada,Exports of goods and services (% of GDP),31.849961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,Turkey,2020,9353.0,1786.51710,8638.739038,1.36748,13.148,TUR,1.85984,0.97039,133756.000000,NaN,15.212980,NaN,NaN,NaN
236,Ukraine,2020,1654.0,846.24850,3709.769287,0.40318,9.475,UKR,-3.75280,-0.60161,-8580.000000,NaN,24.254510,Ukraine,Exports of goods and services (% of GDP),38.821646
237,United States,2020,470131.0,4451.78120,64411.373178,3.42467,8.055,USA,-2.21347,0.96900,848728.187500,7.402097,19.553050,United States,Exports of goods and services (% of GDP),10.202126
238,Uruguay,2020,94.0,813.23790,15789.685742,0.67468,10.413,URY,-7.38010,0.01974,-2530.000000,NaN,15.237220,Uruguay,Exports of goods and services (% of GDP),25.208631


In [264]:
new_panel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 16 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Origin                                           240 non-null    object 
 1   Year                                             240 non-null    int64  
 2   Patent_Applications                              240 non-null    float64
 3   Researchers in R&D (per million people)          240 non-null    float64
 4   GDP per Capita (USD)                             240 non-null    float64
 5   Research and development expenditure (% of GDP)  240 non-null    float64
 6   Unemployemnt total (% of total labor force)      240 non-null    float64
 7   Country Code                                     240 non-null    object 
 8   gdp_growth                                       240 non-null    float64
 9   pop_growth                      

In [251]:
export_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1158 entries, 0 to 1157
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country Name      1158 non-null   object 
 1   Indicator Name    1158 non-null   object 
 2   Origin            1014 non-null   object 
 3   Year              1158 non-null   int64  
 4   Export_Share_GDP  1034 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 45.4+ KB


In [252]:
panel_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 13 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Origin                                           240 non-null    object 
 1   Year                                             240 non-null    int64  
 2   Patent_Applications                              240 non-null    float64
 3   Researchers in R&D (per million people)          240 non-null    float64
 4   GDP per Capita (USD)                             240 non-null    float64
 5   Research and development expenditure (% of GDP)  240 non-null    float64
 6   Unemployemnt total (% of total labor force)      240 non-null    float64
 7   Country Code                                     240 non-null    object 
 8   gdp_growth                                       240 non-null    float64
 9   pop_growth                      

In [248]:
new_panel = panel_merged.merge(
    export_long[['Origin', 'Year', 'Export_Share_GDP']],
    on=['Origin', 'Year'],
    how='left'
)


In [249]:
new_panel

,Origin,Year,Patent_Applications,Researchers in R&D (per million people),GDP per Capita (USD),Research and development expenditure (% of GDP),Unemployemnt total (% of total labor force),Country Code,gdp_growth,pop_growth,net_flow,edu_exp,stem_graduates,Export_Share_GDP
0,Argentina,2015,856.0,1231.48660,13679.626498,0.62262,7.579,ARG,2.73116,1.07800,NaN,NaN,NaN,NaN
1,Austria,2015,11261.0,5074.43000,43915.228021,3.04969,5.802,AUT,1.01450,1.12099,50254.347656,5.848230,29.271710,NaN
2,Belgium,2015,8979.0,4741.86040,40893.804538,2.42817,8.482,BEL,2.04146,0.57945,42932.000000,7.123384,17.382830,NaN
3,Bulgaria,2015,467.0,1939.14420,7078.860323,0.94947,9.143,BGR,3.39755,-0.63807,-12491.000000,NaN,20.793081,NaN
4,Canada,2015,23258.0,4582.37940,43594.194105,1.69359,6.945,CAN,0.64997,0.76030,121634.000000,5.815642,20.027500,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,Turkey,2020,9353.0,1786.51710,8638.739038,1.36748,13.148,TUR,1.85984,0.97039,133756.000000,NaN,15.212980,NaN
236,Ukraine,2020,1654.0,846.24850,3709.769287,0.40318,9.475,UKR,-3.75280,-0.60161,-8580.000000,NaN,24.254510,NaN
237,United States,2020,470131.0,4451.78120,64411.373178,3.42467,8.055,USA,-2.21347,0.96900,848728.187500,7.402097,19.553050,NaN
238,Uruguay,2020,94.0,813.23790,15789.685742,0.67468,10.413,URY,-7.38010,0.01974,-2530.000000,NaN,15.237220,NaN


In [250]:
new_panel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 14 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Origin                                           240 non-null    object 
 1   Year                                             240 non-null    int64  
 2   Patent_Applications                              240 non-null    float64
 3   Researchers in R&D (per million people)          240 non-null    float64
 4   GDP per Capita (USD)                             240 non-null    float64
 5   Research and development expenditure (% of GDP)  240 non-null    float64
 6   Unemployemnt total (% of total labor force)      240 non-null    float64
 7   Country Code                                     240 non-null    object 
 8   gdp_growth                                       240 non-null    float64
 9   pop_growth                      

In [265]:
print(len(new_panel['Origin'].unique()))

40
